<a href="https://colab.research.google.com/github/ancestor9/Gyeongbok-AI-study/blob/main/ancestor9/LGAI_EXAONE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **[Huggingface_LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct](https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct)**

In [ ]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. 모델 초기화
model_name = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,  # 메모리 절약을 위해 bfloat16 사용
    trust_remote_code=True,
    device_map="auto"  # 자동 장치 배치 활성화
)  # .to("cuda") 제거


tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2. Gradio 채팅 인터페이스 함수
def predict(message, history):
    # 시스템 메시지 초기화
    messages = [{"role": "system", "content": "You are EXAONE model from LG AI Research, a helpful assistant."}]

    # 대화 기록 처리
    for user_msg, assistant_msg in history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": assistant_msg})
    messages.append({"role": "user", "content": message})

    # 로컬 모델 추론
    try:
        input_ids = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to("cuda")

        outputs = model.generate(
            input_ids,
            max_new_tokens=128,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id
        )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # 생성된 응답에서 최신 답변만 추출
        response = response.split("assistant\n")[-1].strip()

    except Exception as e:
        response = f"Error: {str(e)}"

    return response

# 3. Gradio 인터페이스 설정
demo = gr.ChatInterface(
    predict,
    title="EXAONE 챗봇",
    description="LG AI Research의 EXAONE 모델 기반 챗봇",
    theme=gr.themes.Soft()
)

# 4. 서버 실행
if __name__ == "__main__":
    demo.launch(share=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://587760049054339e10.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
